# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/SVM

/content/drive/MyDrive/SVM


In [ ]:
%ls

Datasets/                  RAVTESS_MFCC_Observed.csv
RAVDESS_MFCC_Observed.csv  TESS_MFCC_Observed.csv


### Import Libraries

Import necessary libraries

In [ ]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','surprised','happy','sad','neutral']
 
##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']



#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [ ]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result = np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [ ]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [ ]:
def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    data = dataset_options()
    paths = []
    if data['ravdess']:
        paths.append("Datasets/RAVDESS/*/Actor_*/*.wav")
    elif data['ravdess_speech']:
        paths.append("Datasets/RAVDESS/Speech/Actor_*/*.wav")
    elif data['ravdess_song']:
        paths.append("Datasets/RAVDESS/Song/Actor_*/*.wav")

    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion according to filename. dictionary emotions is defined above.
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            iterations = 1
            while(iterations<observed_iterations):
              iterations = iterations+1
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("Datasets/TESS/*AF_*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
            
    return {"X":x,"y":y}

In [ ]:
start_time = time.time()
iterations = 1

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 1984.4953927993774 seconds ---


In [ ]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [ ]:
X.shape, y.shape

((5252, 40), (5252, 1))

In [ ]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [ ]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-582.839966,31.625124,-26.554098,8.023311,-18.265991,-10.929363,-11.114575,-18.726753,-10.231754,-0.531983,...,1.099666,-1.430102,-5.383887,-3.371233,-0.047072,2.375369,0.315157,2.039546,6.139904,neutral
1,-594.304871,37.779697,-26.957352,13.258965,-20.494688,-9.329007,-13.782453,-16.953814,-10.334120,-3.184894,...,1.277685,-1.734711,-5.414732,-3.810104,0.133354,2.128390,0.477521,1.827881,6.518322,neutral
2,-557.573547,32.991276,-30.908392,7.254938,-22.056374,-9.388789,-13.758408,-17.887318,-10.950238,-1.501468,...,7.895620,-0.981356,-9.882403,-4.660424,-2.089421,-1.277220,-1.556431,7.738025,9.197818,calm
3,-561.938354,34.059616,-34.383072,6.717714,-27.443283,-11.699245,-12.053927,-23.312895,-11.367053,-3.053010,...,4.855764,-3.677525,-10.760934,-4.159203,-2.416040,-1.261963,1.011953,9.588315,10.089292,calm
4,-574.555298,35.277279,-21.822271,13.926046,-16.524294,-8.877282,-11.371426,-19.489376,-5.584670,-1.907575,...,0.755291,-0.458310,-5.396890,-5.797887,-0.661944,1.487520,-0.044380,0.520410,5.811005,neutral


## Shuffling data

In [ ]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [ ]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")